In [20]:
import re

from langchain import hub
from langchain.agents.agent import AgentExecutor
from langchain.agents.react.agent import create_react_agent
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_openai.chat_models.base import ChatOpenAI

sql_prompt = hub.pull("langchain-ai/sql-agent-system-prompt")
react_prompt = hub.pull("hwchase17/react")
combined_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", sql_prompt.messages[0].prompt.template + "\n" + react_prompt.template),
    ]
)
print(combined_prompt.messages)

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'dialect', 'input', 'tool_names', 'tools', 'top_k'], input_types={}, partial_variables={}, template='You are an agent designed to interact with a SQL database.\nGiven an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.\nUnless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.\nYou can order the results by a relevant column to return the most interesting examples in the database.\nNever query for all the columns from a specific table, only ask for the relevant columns given the question.\nYou have access to tools for interacting with the database.\nOnly use the below tools. Only use the information returned by the below tools to construct your final answer.\nYou MUST double check your query before executing it. If you get an error while executing a

In [25]:
sql_prompt.messages[0].prompt.template

'You are an agent designed to interact with a SQL database.\nGiven an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.\nUnless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.\nYou can order the results by a relevant column to return the most interesting examples in the database.\nNever query for all the columns from a specific table, only ask for the relevant columns given the question.\nYou have access to tools for interacting with the database.\nOnly use the below tools. Only use the information returned by the below tools to construct your final answer.\nYou MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.\n\nDO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.\n\nTo start you should ALWAYS look at the tables in the data

In [8]:
[
    *sql_prompt.messages,
]

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['dialect', 'top_k'], input_types={}, partial_variables={}, template='You are an agent designed to interact with a SQL database.\nGiven an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.\nUnless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.\nYou can order the results by a relevant column to return the most interesting examples in the database.\nNever query for all the columns from a specific table, only ask for the relevant columns given the question.\nYou have access to tools for interacting with the database.\nOnly use the below tools. Only use the information returned by the below tools to construct your final answer.\nYou MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.\n\nDO NOT m

In [16]:
x = sql_prompt.messages[0].prompt.template
print(x)

You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see 

In [18]:
x = react_prompt.template
print(x)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [ ]:
[
    SystemMessagePromptTemplate(
        prompt=PromptTemplate(
            input_variables=["dialect", "top_k"],
            input_types={},
            partial_variables={},
            template="You are an agent designed to interact with a SQL database.\nGiven an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.\nUnless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.\nYou can order the results by a relevant column to return the most interesting examples in the database.\nNever query for all the columns from a specific table, only ask for the relevant columns given the question.\nYou have access to tools for interacting with the database.\nOnly use the below tools. Only use the information returned by the below tools to construct your final answer.\nYou MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.\n\nDO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.\n\nTo start you should ALWAYS look at the tables in the database to see what you can query.\nDo NOT skip this step.\nThen you should query the schema of the most relevant tables.",
        ),
        additional_kwargs={},
    )
]

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
import re
from langchain.document_loaders import UnstructuredURLLoader
from langchain_core.documents.base import Document
from typing import List

url = "https://lilianweng.github.io/posts/2023-06-23-agent/"


def webloader(url: str) -> List[Document]:
    """Load the content of a website from url to text."""
    docs = WebBaseLoader(url).load()
    docs = [re.sub(r"\n{3,}", r"\n\n", doc.page_content) for doc in docs]
    return docs


print(webloader(url))

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
import re
from langchain.document_loaders import UnstructuredURLLoader

url = "https://lilianweng.github.io/posts/2023-06-23-agent/"


print(UnstructuredURLLoader(urls=[url]).load())

In [ ]:
from docling.document_converter import DocumentConverter

source = "https://lilianweng.github.io/posts/2023-06-23-agent/"  # PDF path or URL
converter = DocumentConverter()
result = converter.convert(source)
print(result.document.export_to_markdown())

In [ ]:
import logging
import time
from pathlib import Path
from docling_core.types.doc import ImageRefMode, PictureItem, TableItem
from docling.datamodel.base_models import FigureElement, InputFormat, Table
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.datamodel.base_models import FigureElement, InputFormat, Table
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.datamodel.base_models import FigureElement, InputFormat, Table
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.document_converter import DocumentConverter, PdfFormatOption

_log = logging.getLogger(__name__)

IMAGE_RESOLUTION_SCALE = 2.0


def main():
    logging.basicConfig(level=logging.INFO)

    input_doc_path = Path("./pdfs/Masterpieces.pdf")
    output_dir = Path("scratch")

    # Important: For operating with page images, we must keep them, otherwise the DocumentConverter
    # will destroy them for cleaning up memory.
    # This is done by setting PdfPipelineOptions.images_scale, which also defines the scale of images.
    # scale=1 correspond of a standard 72 DPI image
    # The PdfPipelineOptions.generate_* are the selectors for the document elements which will be enriched
    # with the image field
    pipeline_options = PdfPipelineOptions()
    pipeline_options.images_scale = IMAGE_RESOLUTION_SCALE
    pipeline_options.generate_page_images = True
    pipeline_options.generate_table_images = True
    pipeline_options.generate_picture_images = True

    doc_converter = DocumentConverter(format_options={InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)})

    start_time = time.time()

    conv_res = doc_converter.convert(input_doc_path)

    output_dir.mkdir(parents=True, exist_ok=True)
    doc_filename = conv_res.input.file.stem

    # Save page images
    for page_no, page in conv_res.document.pages.items():
        page_no = page.page_no
        page_image_filename = output_dir / f"{doc_filename}-{page_no}.png"
        with page_image_filename.open("wb") as fp:
            page.image.pil_image.save(fp, format="PNG")

    # Save images of figures and tables
    table_counter = 0
    picture_counter = 0
    for element, _level in conv_res.document.iterate_items():
        if isinstance(element, TableItem):
            table_counter += 1
            element_image_filename = output_dir / f"{doc_filename}-table-{table_counter}.png"
            with element_image_filename.open("wb") as fp:
                element.image.pil_image.save(fp, "PNG")

        if isinstance(element, PictureItem):
            picture_counter += 1
            element_image_filename = output_dir / f"{doc_filename}-picture-{picture_counter}.png"
            with element_image_filename.open("wb") as fp:
                element.image.pil_image.save(fp, "PNG")

    # Save markdown with embedded pictures
    content_md = conv_res.document.export_to_markdown(image_mode=ImageRefMode.EMBEDDED)
    md_filename = output_dir / f"{doc_filename}-with-images.md"
    with md_filename.open("w") as fp:
        fp.write(content_md)

    end_time = time.time() - start_time

    _log.info(f"Document converted and figures exported in {end_time:.2f} seconds.")


if __name__ == "__main__":
    main()

In [1]:
from TextToSQL.react import text_to_sql_react
from TextToSQL.simple import text_to_sql


questions = [
    "What are the 10 most expensive albums",
]

for i, question in enumerate(questions, 1):
    print(f"\nQuestion {i}:\n{question}")
    # print(f"\nSimple Response {i}:")
    # response = text_to_sql(question)
    # print(response)
    print(f"\nReAct Response {i}:")
    print(text_to_sql_react(question))
    if i < len(questions):
        print("---")


Question 1:
What are the 10 most expensive albums

ReAct Response 1:
{'messages': [HumanMessage(content='What are the 10 most expensive albums', additional_kwargs={}, response_metadata={}, id='2cc2df89-6b1d-4e0a-9702-7c39cc58b577'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Kh5cWG8zxP0v5SG86iJDMPJU', 'function': {'arguments': '{}', 'name': 'sql_db_list_tables'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 554, 'total_tokens': 566, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-c50b9495-2245-467c-977c-dcdd6483aefa-0', tool_calls=[{'name': 'sql_db_list_tables', 'args': {}, 'id': 'call_Kh5cWG8zxP0v5

In [3]:
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

response = {
    "messages": [
        HumanMessage(content="What are the 10 most expensive albums", additional_kwargs={}, response_metadata={}, id="2cc2df89-6b1d-4e0a-9702-7c39cc58b577"),
        AIMessage(content="", additional_kwargs={"tool_calls": [{"id": "call_Kh5cWG8zxP0v5SG86iJDMPJU", "function": {"arguments": "{}", "name": "sql_db_list_tables"}, "type": "function"}], "refusal": None}, response_metadata={"token_usage": {"completion_tokens": 12, "prompt_tokens": 554, "total_tokens": 566, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_tokens_details": {"audio_tokens": 0, "cached_tokens": 0}}, "model_name": "gpt-4o-mini-2024-07-18", "system_fingerprint": "fp_0ba0d124f1", "finish_reason": "tool_calls", "logprobs": None}, id="run-c50b9495-2245-467c-977c-dcdd6483aefa-0", tool_calls=[{"name": "sql_db_list_tables", "args": {}, "id": "call_Kh5cWG8zxP0v5SG86iJDMPJU", "type": "tool_call"}], usage_metadata={"input_tokens": 554, "output_tokens": 12, "total_tokens": 566, "input_token_details": {"audio": 0, "cache_read": 0}, "output_token_details": {"audio": 0, "reasoning": 0}}),
        ToolMessage(content="Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track", name="sql_db_list_tables", id="7c3616c8-00d7-4bdc-925f-d2d430c39684", tool_call_id="call_Kh5cWG8zxP0v5SG86iJDMPJU"),
        AIMessage(
            content="",
            additional_kwargs={"tool_calls": [{"id": "call_nEc605jmBzSdDMlaU3xAVe3b", "function": {"arguments": '{"table_names": "Album"}', "name": "sql_db_schema"}, "type": "function"}, {"id": "call_CtshKLneChZdZAmF6yLiKvK9", "function": {"arguments": '{"table_names": "InvoiceLine"}', "name": "sql_db_schema"}, "type": "function"}], "refusal": None},
            response_metadata={"token_usage": {"completion_tokens": 49, "prompt_tokens": 600, "total_tokens": 649, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_tokens_details": {"audio_tokens": 0, "cached_tokens": 0}}, "model_name": "gpt-4o-mini-2024-07-18", "system_fingerprint": "fp_0ba0d124f1", "finish_reason": "tool_calls", "logprobs": None},
            id="run-dc477549-6a60-4375-9967-49ede0d51c5f-0",
            tool_calls=[{"name": "sql_db_schema", "args": {"table_names": "Album"}, "id": "call_nEc605jmBzSdDMlaU3xAVe3b", "type": "tool_call"}, {"name": "sql_db_schema", "args": {"table_names": "InvoiceLine"}, "id": "call_CtshKLneChZdZAmF6yLiKvK9", "type": "tool_call"}],
            usage_metadata={"input_tokens": 600, "output_tokens": 49, "total_tokens": 649, "input_token_details": {"audio": 0, "cache_read": 0}, "output_token_details": {"audio": 0, "reasoning": 0}},
        ),
        ToolMessage(content='\nCREATE TABLE "Album" (\n\t"AlbumId" INTEGER NOT NULL, \n\t"Title" NVARCHAR(160) NOT NULL, \n\t"ArtistId" INTEGER NOT NULL, \n\tPRIMARY KEY ("AlbumId"), \n\tFOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")\n)\n\n/*\n3 rows from Album table:\nAlbumId\tTitle\tArtistId\n1\tFor Those About To Rock We Salute You\t1\n2\tBalls to the Wall\t2\n3\tRestless and Wild\t2\n*/', name="sql_db_schema", id="6eeb427f-c0da-4bd8-96a5-48bcc460fc93", tool_call_id="call_nEc605jmBzSdDMlaU3xAVe3b"),
        ToolMessage(content='\nCREATE TABLE "InvoiceLine" (\n\t"InvoiceLineId" INTEGER NOT NULL, \n\t"InvoiceId" INTEGER NOT NULL, \n\t"TrackId" INTEGER NOT NULL, \n\t"UnitPrice" NUMERIC(10, 2) NOT NULL, \n\t"Quantity" INTEGER NOT NULL, \n\tPRIMARY KEY ("InvoiceLineId"), \n\tFOREIGN KEY("TrackId") REFERENCES "Track" ("TrackId"), \n\tFOREIGN KEY("InvoiceId") REFERENCES "Invoice" ("InvoiceId")\n)\n\n/*\n3 rows from InvoiceLine table:\nInvoiceLineId\tInvoiceId\tTrackId\tUnitPrice\tQuantity\n1\t1\t2\t0.99\t1\n2\t1\t4\t0.99\t1\n3\t2\t6\t0.99\t1\n*/', name="sql_db_schema", id="f6e2d25f-4a05-497a-81ce-405a2bd5ea85", tool_call_id="call_CtshKLneChZdZAmF6yLiKvK9"),
        AIMessage(
            content="",
            additional_kwargs={"tool_calls": [{"id": "call_QjnUL9BYncs69IgL2AtMStgs", "function": {"arguments": '{"query":"SELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \\nFROM Album a \\nJOIN InvoiceLine il ON a.AlbumId = il.TrackId \\nGROUP BY a.AlbumId, a.Title \\nORDER BY TotalPrice DESC \\nLIMIT 10;"}', "name": "sql_db_query_checker"}, "type": "function"}], "refusal": None},
            response_metadata={"token_usage": {"completion_tokens": 74, "prompt_tokens": 980, "total_tokens": 1054, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_tokens_details": {"audio_tokens": 0, "cached_tokens": 0}}, "model_name": "gpt-4o-mini-2024-07-18", "system_fingerprint": "fp_0ba0d124f1", "finish_reason": "tool_calls", "logprobs": None},
            id="run-69c18b6f-02de-4c9c-8559-92ffc719e958-0",
            tool_calls=[{"name": "sql_db_query_checker", "args": {"query": "SELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \nFROM Album a \nJOIN InvoiceLine il ON a.AlbumId = il.TrackId \nGROUP BY a.AlbumId, a.Title \nORDER BY TotalPrice DESC \nLIMIT 10;"}, "id": "call_QjnUL9BYncs69IgL2AtMStgs", "type": "tool_call"}],
            usage_metadata={"input_tokens": 980, "output_tokens": 74, "total_tokens": 1054, "input_token_details": {"audio": 0, "cache_read": 0}, "output_token_details": {"audio": 0, "reasoning": 0}},
        ),
        ToolMessage(content="```sql\nSELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \nFROM Album a \nJOIN InvoiceLine il ON a.AlbumId = il.TrackId \nGROUP BY a.AlbumId, a.Title \nORDER BY TotalPrice DESC \nLIMIT 10;\n```", name="sql_db_query_checker", id="be74db7a-034e-49b0-923e-a8543235c99c", tool_call_id="call_QjnUL9BYncs69IgL2AtMStgs"),
        AIMessage(
            content="",
            additional_kwargs={"tool_calls": [{"id": "call_feyASAXqOgFWas6rLxGQ7TPz", "function": {"arguments": '{"query":"SELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \\nFROM Album a \\nJOIN InvoiceLine il ON a.AlbumId = il.TrackId \\nGROUP BY a.AlbumId, a.Title \\nORDER BY TotalPrice DESC \\nLIMIT 10;"}', "name": "sql_db_query"}, "type": "function"}], "refusal": None},
            response_metadata={"token_usage": {"completion_tokens": 73, "prompt_tokens": 1122, "total_tokens": 1195, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_tokens_details": {"audio_tokens": 0, "cached_tokens": 1024}}, "model_name": "gpt-4o-mini-2024-07-18", "system_fingerprint": "fp_0ba0d124f1", "finish_reason": "tool_calls", "logprobs": None},
            id="run-93c2f722-d420-451c-93db-58f2853d99fd-0",
            tool_calls=[{"name": "sql_db_query", "args": {"query": "SELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \nFROM Album a \nJOIN InvoiceLine il ON a.AlbumId = il.TrackId \nGROUP BY a.AlbumId, a.Title \nORDER BY TotalPrice DESC \nLIMIT 10;"}, "id": "call_feyASAXqOgFWas6rLxGQ7TPz", "type": "tool_call"}],
            usage_metadata={"input_tokens": 1122, "output_tokens": 73, "total_tokens": 1195, "input_token_details": {"audio": 0, "cache_read": 1024}, "output_token_details": {"audio": 0, "reasoning": 0}},
        ),
        ToolMessage(content="[('Balls to the Wall', 1.98), ('Warner 25 Anos', 1.98), ('Plays Metallica By Four Cellos', 1.98), ('The Best Of Buddy Guy - The Millenium Collection', 1.98), ('Carnaval 2001', 1.98), ('The Essential Miles Davis [Disc 1]', 1.98), ('The Battle Rages On', 1.98), ('Roda De Funk', 1.98), ('Demorou...', 1.98), ('Motley Crue Greatest Hits', 1.98)]", name="sql_db_query", id="af24e4f9-ec4b-4e53-b4c0-9b413d209df6", tool_call_id="call_feyASAXqOgFWas6rLxGQ7TPz"),
        AIMessage(
            content="The 10 most expensive albums based on total sales are:\n\n1. **Balls to the Wall** - $1.98\n2. **Warner 25 Anos** - $1.98\n3. **Plays Metallica By Four Cellos** - $1.98\n4. **The Best Of Buddy Guy - The Millenium Collection** - $1.98\n5. **Carnaval 2001** - $1.98\n6. **The Essential Miles Davis [Disc 1]** - $1.98\n7. **The Battle Rages On** - $1.98\n8. **Roda De Funk** - $1.98\n9. **Demorou...** - $1.98\n10. **Motley Crue Greatest Hits** - $1.98\n\nAll these albums have a total price of $1.98.",
            additional_kwargs={"refusal": None},
            response_metadata={"token_usage": {"completion_tokens": 188, "prompt_tokens": 1335, "total_tokens": 1523, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_tokens_details": {"audio_tokens": 0, "cached_tokens": 1024}}, "model_name": "gpt-4o-mini-2024-07-18", "system_fingerprint": "fp_0ba0d124f1", "finish_reason": "stop", "logprobs": None},
            id="run-23d620be-10f0-4cf4-b142-038ac3539dd8-0",
            usage_metadata={"input_tokens": 1335, "output_tokens": 188, "total_tokens": 1523, "input_token_details": {"audio": 0, "cache_read": 1024}, "output_token_details": {"audio": 0, "reasoning": 0}},
        ),
    ]
}

In [5]:
response["messages"]

[HumanMessage(content='What are the 10 most expensive albums', additional_kwargs={}, response_metadata={}, id='2cc2df89-6b1d-4e0a-9702-7c39cc58b577'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Kh5cWG8zxP0v5SG86iJDMPJU', 'function': {'arguments': '{}', 'name': 'sql_db_list_tables'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 554, 'total_tokens': 566, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-c50b9495-2245-467c-977c-dcdd6483aefa-0', tool_calls=[{'name': 'sql_db_list_tables', 'args': {}, 'id': 'call_Kh5cWG8zxP0v5SG86iJDMPJU', 'type': 'tool_call'}], usage_metadata={'input_tokens': 554, 'output_

In [10]:
answer = next(msg for msg in reversed(response["messages"]) if isinstance(msg, AIMessage))
print(answer.content)

The 10 most expensive albums based on total sales are:

1. **Balls to the Wall** - $1.98
2. **Warner 25 Anos** - $1.98
3. **Plays Metallica By Four Cellos** - $1.98
4. **The Best Of Buddy Guy - The Millenium Collection** - $1.98
5. **Carnaval 2001** - $1.98
6. **The Essential Miles Davis [Disc 1]** - $1.98
7. **The Battle Rages On** - $1.98
8. **Roda De Funk** - $1.98
9. **Demorou...** - $1.98
10. **Motley Crue Greatest Hits** - $1.98

All these albums have a total price of $1.98.


In [15]:
result = next(msg for msg in reversed(response["messages"]) if isinstance(msg, ToolMessage) and msg.name == "sql_db_query")
print(result.content)

[('Balls to the Wall', 1.98), ('Warner 25 Anos', 1.98), ('Plays Metallica By Four Cellos', 1.98), ('The Best Of Buddy Guy - The Millenium Collection', 1.98), ('Carnaval 2001', 1.98), ('The Essential Miles Davis [Disc 1]', 1.98), ('The Battle Rages On', 1.98), ('Roda De Funk', 1.98), ('Demorou...', 1.98), ('Motley Crue Greatest Hits', 1.98)]


In [14]:
query = next(msg for msg in reversed(response["messages"]) if isinstance(msg, ToolMessage) and msg.name == "sql_db_query_checker")
print(query.content)

```sql
SELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice 
FROM Album a 
JOIN InvoiceLine il ON a.AlbumId = il.TrackId 
GROUP BY a.AlbumId, a.Title 
ORDER BY TotalPrice DESC 
LIMIT 10;
```
